In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from utils.imports import *

/home/twofyw/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Normalization

In [5]:
from data_reader import calc_mean_std_par_dir

In [7]:
subdir = '../data/SRAD2018_TRAIN_001/RAD_206482464212530/'

In [15]:
par_dir = Path('../data/SRAD2018_TRAIN_001')
subdirs = [o for o in par_dir.iterdir() if o.is_dir()]

In [ ]:
subdir

In [22]:
%%time
means, stds = zip(*[calc_mean_std_subdir(o) for o in subdirs])

In [23]:
means, stds

((0.9820557,
  0.7564683,
  0.968596,
  0.9650422,
  0.98231244,
  0.7932429,
  0.9655625,
  0.97146374,
  0.9070648,
  0.7613075),
 (0.12885971,
  0.3993739,
  0.16909033,
  0.17655694,
  0.12847538,
  0.39483172,
  0.17846707,
  0.16026804,
  0.28292444,
  0.41495243))

In [6]:
mean , std = np.mean(stds), np.mean(means)

NameError: name 'stds' is not defined

In [7]:
mean, std = calc_mean_std_par_dir('../data/SRAD2018_TRAIN_002')

KeyboardInterrupt: 

In [ ]:
mean, std

In [39]:
fn = 'stat.csv'
with open(fn, 'w') as f:
        np.array([mean, std]).tofile(f, ',')

In [40]:
!cat stat.csv

0.27102503180503845,0.8787143230438232

## SequenceGenerator

In [19]:
sess = tf.InteractiveSession()

In [20]:
shape = (61, 501, 501, 1)
def parser_train(serialized_example):
    features, sequence_features = tf.parse_single_sequence_example(
        serialized_example, context_features={
            'time_stamp': tf.FixedLenFeature([], tf.string),
        }, sequence_features={
            "raw_png": tf.FixedLenSequenceFeature([], dtype=tf.string)
        })
    
    x = tf.map_fn(tf.image.decode_png, sequence_features['raw_png'], dtype=tf.uint8,
                back_prop=False, swap_memory=False, infer_shape=False)
    x = tf.cast(x, tf.float32)
    x /= 255
    x.set_shape(shape)
    return x, features['time_stamp']

In [21]:
from utils.transforms import *

sz = 128
nt = 10
stats = np.fromfile('stat.csv', sep=',')

class Slice(Transform):
    """ Return a slice of the images
    
    Arguments:
    The same as the built-in function slice
    """
    def __init__(self, *args, **kwargs):
        self.slice = slice(*args, **kwargs)
        super().__init__(TfmType.NO)
        
    def do_transform(self, x, is_y):
        return x[self.slice]
    
aug_tfms = [Slice(nt)]
tfm, _ = tfms_from_stats(stats, sz, aug_tfms=aug_tfms, crop_type=CropType.NO)
tfm

[<utils.transforms.Scale object at 0x7fcb5152e4e0>, <__main__.Slice object at 0x7fcb5152e438>, <utils.transforms.Normalize object at 0x7fcb5152e358>]

In [22]:
bs = 3

filenames = tf.placeholder(tf.string, name='tfrecords')
training_filenames = '../data/tfrecords/train_1.tfrecords'
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parser_train).map(tfm).batch(bs).repeat()
# dataset = dataset.prefetch()
iterator = dataset.make_initializable_iterator()
iterator.initializer.run({filenames: training_filenames})
x, time_stamp = iterator.get_next()

In [26]:
sample = x[0]

In [23]:
x.shape

TensorShape([Dimension(None), Dimension(10), Dimension(128), Dimension(128), Dimension(1)])

In [24]:
x.eval().shape

(3, 10, 128, 128, 1)

## Prednet

In [29]:
from models.prednet import PredNet

In [13]:
from tensorflow.keras import backend as K
import tensorflow.keras as keras

In [30]:
# n_channels, im_height, im_width = (3, 128, 160)
n_channels, im_height, im_width = (1, sz, sz)
input_shape = (n_channels, im_height, im_width) if K.image_data_format() == 'channels_first' else (im_height, im_width, n_channels)
stack_sizes = (n_channels, 48, 96, 192)
R_stack_sizes = stack_sizes
A_filt_sizes = (3, 3, 3)
Ahat_filt_sizes = (3, 3, 3, 3)
R_filt_sizes = (3, 3, 3, 3)
prednet = PredNet(stack_sizes, R_stack_sizes,
                  A_filt_sizes, Ahat_filt_sizes, R_filt_sizes,
                  output_mode='error', return_sequences=True, cell=tf.contrib.rnn.LSTMCell)

data_format:	 channels_last
channel_axis:	 -1
row_axis:	 -3
column_axis:	 -2


In [31]:
prednet.build(sample.shape)

In [33]:
prednet.call(sample, initial_state=prednet.get_initial_state(x))

input_spec [InputSpec(shape=(10, 128, 128, 1), ndim=4)]
> /home/twofyw/pred-rain/yang/models/prednet.py(238)call()
    236             a = K.switch(t >= self.t_extrap, states[-2], a)  # if past self.extrap_start_time, the previous prediction will be treated as the actual
    237         set_trace()
--> 238         c = []
    239         r = []
    240         e = []

ipdb> a_backup = a
ipdb> c
> /home/twofyw/pred-rain/yang/models/prednet.py(305)call()
    303         set_trace()
    304 
--> 305         return output, states
    306 
    307     def get_config(self):

ipdb> i0
*** NameError: name 'i0' is not defined
ipdb> i0 = self.conv_layers['i'][0]
ipdb> p i0(a_backup).shape
*** ValueError: Input 0 of layer conv2d_23 is incompatible with the layer: expected axis -1 of input shape to have value 51 but received input with shape [10, 128, 128, 1]
ipdb> i0
ipdb> i0.__dict__
{'_setattr_tracking': True, 'trainable': True, 'stateful': False, 'built': True, 'input_spec': InputSpec(ndim=4, a

BdbQuit: 

In [62]:
nt = 10
inputs = keras.layers.Input(shape=(nt,) + input_shape)
model = keras.Model(inputs=x, outputs=)

In [63]:
model.compile(loss='mean_absolute_error', optimizer='adam')

In [65]:
model.predict(x, steps=10)

AttributeError: 'Model' object has no attribute '_output_tensor_cache'

In [25]:
K.image_data_format()

'channels_last'

In [50]:
sample = x[0]

In [54]:
from models.prednet import PredNet
prednet = PredNet(stack_sizes, R_stack_sizes,
                  A_filt_sizes, Ahat_filt_sizes, R_filt_sizes,
                  output_mode='error', return_sequences=True, cell=tf.contrib.rnn.LSTMCell)
prednet.build(sample.shape)
prednet.get_initial_state(x)

data_format:	 channels_last
channel_axis:	 -1
row_axis:	 -3
column_axis:	 -2
input_spec [InputSpec(shape=(10, 128, 128, 1), ndim=4)]


[<tf.Tensor 'Reshape_84:0' shape=(?, 128, 128, 1) dtype=float32>,
 <tf.Tensor 'Reshape_85:0' shape=(?, 64, 64, 48) dtype=float32>,
 <tf.Tensor 'Reshape_86:0' shape=(?, 32, 32, 96) dtype=float32>,
 <tf.Tensor 'Reshape_87:0' shape=(?, 16, 16, 192) dtype=float32>,
 <tf.Tensor 'Reshape_88:0' shape=(?, 128, 128, 1) dtype=float32>,
 <tf.Tensor 'Reshape_89:0' shape=(?, 64, 64, 48) dtype=float32>,
 <tf.Tensor 'Reshape_90:0' shape=(?, 32, 32, 96) dtype=float32>,
 <tf.Tensor 'Reshape_91:0' shape=(?, 16, 16, 192) dtype=float32>,
 <tf.Tensor 'Reshape_92:0' shape=(?, 128, 128, 2) dtype=float32>,
 <tf.Tensor 'Reshape_93:0' shape=(?, 64, 64, 96) dtype=float32>,
 <tf.Tensor 'Reshape_94:0' shape=(?, 32, 32, 192) dtype=float32>,
 <tf.Tensor 'Reshape_95:0' shape=(?, 16, 16, 384) dtype=float32>]

In [55]:
prednet.compute_output_shape(sample.shape)

(Dimension(10), Dimension(128), 4)

In [59]:
y = prednet(sample, prednet.get_initial_state(x))

input_spec [InputSpec(shape=(10, 128, 128, 1), ndim=4)]


In [ ]:
model.compile(loss='mean_absolute_error', optimizer='adam')

In [60]:
y_np = [o.eval() for o in y]

FailedPreconditionError: Error while reading resource variable pred_net_15/layer_a_0/kernel from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/pred_net_15/layer_a_0/kernel)
	 [[Node: pred_net_15_3/Conv2D_17/ReadVariableOp = ReadVariableOp[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](pred_net_15/layer_a_0/kernel)]]

Caused by op 'pred_net_15_3/Conv2D_17/ReadVariableOp', defined at:
  File "/home/twofyw/miniconda3/envs/tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/twofyw/miniconda3/envs/tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/twofyw/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/twofyw/miniconda3/envs/tf/lib/python3.6/site-packages/traitlets/config/applicatio

In [59]:
prednet.build(x.shape)

## Keras

In [134]:
import tensorflow.keras as keras

In [136]:
rnn = keras.layers.RNN

In [138]:
rnn??

In [70]:
from tensorflow.keras.layers import *

In [84]:
a_convlstm_1 = ConvLSTM2D(stack_sizes[1], A_filt_sizes[0], padding='same', activation='hard_sigmoid', )

In [85]:
a_convlstm_1.build(x.shape)

In [94]:
a_convlstm_1.compute_output_shape(x.shape)

TensorShape([Dimension(None), Dimension(128), Dimension(128), Dimension(48)])

# RUN

In [66]:
prednet = PredNet(stack_sizes, R_stack_sizes,
                  A_filt_sizes, Ahat_filt_sizes, R_filt_sizes,
                  output_mode='error', return_sequences=True)

inputs = Input(shape=(nt,) + input_shape)
errors = prednet(inputs)  # errors will be (batch_size, nt, nb_layers)
errors_by_time = TimeDistributed(Dense(1, trainable=False), weights=[layer_loss_weights, np.zeros(1)], trainable=False)(errors)  # calculate weighted error by layer
errors_by_time = Flatten()(errors_by_time)  # will be (batch_size, nt)
final_errors = Dense(1, weights=[time_loss_weights, np.zeros(1)], trainable=False)(errors_by_time)  # weight errors by time
model = Model(inputs=inputs, outputs=final_errors)
model.compile(loss='mean_absolute_error', optimizer='adam')

data_format:	 channels_last
channel_axis:	 -1
row_axis:	 -3
column_axis:	 -2


TypeError: __init__() missing 1 required positional argument: 'cell'